In [226]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [227]:
df = pd.read_csv('test.csv')
dft = pd.read_csv('todasIdeia2.csv')

In [228]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [229]:
valores_nulos = df.isnull().sum()
print(valores_nulos)

full_name              0
registered_year       15
engine_capacity        5
insurance              2
transmission_type      0
kms_driven             0
owner_type             9
fuel_type              0
max_power             13
seats                  3
mileage              101
body_type              0
city                   0
ID                     0
dtype: int64


In [230]:
# SEATS __________________________________________________
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)


# ENGINE __________________________________________________
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')


# KMS __________________________________________________
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

# MAX POWER 
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)

Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]
df['max_power'] = Correct_Max_Power



# MILEAGE __________________________________________________
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40

        consumo_correto.append(valor if pd.notnull(i) else None)

while len(consumo_correto) < len(df):
    consumo_correto.append(None)

df['mileage'] = consumo_correto

media_mileage = df['mileage'].astype(float).mean()
df['mileage'].fillna(media_mileage, inplace=True)

# OWNER TYPE __________________________________________________

# Substituir os valores em branco ('') pela moda ('mode') da coluna 'owner_type'
moda_owner_type = df['owner_type'].mode()[0]  # Calcula a moda
df['owner_type'].replace('', moda_owner_type, inplace=True)

# INSURANCE __________________________________________________
#Deletar "insurance"
df['insurance'] = df['insurance'].str.replace('insurance', '')

#Substituir valores em branco e valores numéricos por 'No Insurance'
df['insurance'] = df['insurance'].apply(lambda x: 'No Insurance' if pd.isnull(x) or pd.api.types.is_numeric_dtype(x) else x)

#Corrigir "Third Party" com espaço extra
df['insurance'] = df['insurance'].str.replace('Third Party ', 'Third Party')


Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2012 Honda Jazz Select Edition


In [231]:
valores_nulos = df.isnull().sum()
print(valores_nulos)

full_name             0
registered_year      15
engine_capacity       5
insurance             0
transmission_type     0
kms_driven            0
owner_type            9
fuel_type             0
max_power            13
seats                 2
mileage               0
body_type             0
city                  0
ID                    0
dtype: int64


In [232]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,mileage,body_type,city,ID
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197,Comprehensive,Manual,53658,First Owner,Petrol,81.8,5.0,18.900000,Hatchback,Mumbai,0
1,2020 Maruti Swift AMT VXI,Feb 2021,1197,Not Available,Automatic,33055,Second Owner,Petrol,81.8,5.0,21.210000,Hatchback,Jaipur,1
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998,Third Party,Manual,90385,Third Owner,Petrol,67.1,5.0,18.200000,Hatchback,Chandigarh,2
3,2010 Maruti Alto LXi,2010,796,Third Party,Manual,110000,First Owner,Petrol,46.3,5.0,19.700000,Hatchback,Delhi,3
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197,Comprehensive,Manual,58000,Second Owner,Petrol,81.8,5.0,17.190000,Hatchback,Delhi,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,NaN,Third Party,Automatic,30000,First Owner,Electric,73.7,5.0,19.520446,Sedan,Pune,3485
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999,Third Party,Manual,64024,First Owner,Petrol,67.0,5.0,19.520446,Hatchback,Bangalore,3486
3487,2017 Honda City i VTEC VX,Dec 2017,1497,Comprehensive,Manual,39000,First Owner,Petrol,117.3,5.0,19.520446,Sedan,Delhi,3487
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199,Comprehensive,Manual,33000,First Owner,Petrol,88.7,5.0,19.520446,Hatchback,Mumbai,3488


In [233]:
#FullName ______________________________________

#ANO
# Extract the year and create a new column
df['ano'] = df['full_name'].str.extract(r'(\d{4})')

# Display the resulting DataFrame
print(df['ano'])

#MARCA
# Extract the first word after the year and create a new column
df['marca'] = df['full_name'].str.extract(r'\d{4}\s(\w+)')

# Print the resulting DataFrame
print(df['marca'])

#MODELO
# Get everything that comes after "marca"
df['modelo'] = df['full_name'].str.extract(r'\d{4}\s\w+\s(.+)')

# Print the resulting DataFrame
print(df['modelo'])

0       2017
1       2020
2       2010
3       2010
4       2018
        ... 
3485    2022
3486    2017
3487    2017
3488    2016
3489    2017
Name: ano, Length: 3490, dtype: object
0       Hyundai
1        Maruti
2        Maruti
3        Maruti
4       Hyundai
         ...   
3485       Tata
3486    Renault
3487      Honda
3488      Honda
3489     Maruti
Name: marca, Length: 3490, dtype: object
0       Grand i10 1.2 Kappa Sportz Option
1                           Swift AMT VXI
2                     Zen Estilo LX BSIII
3                                Alto LXi
4                     i20 Active S Petrol
                      ...                
3485                     Tigor EV XZ Plus
3486                         KWID 1.0 RXT
3487                       City i VTEC VX
3488                    Jazz 1.2 V i VTEC
3489                            Swift VXI
Name: modelo, Length: 3490, dtype: object


In [234]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [235]:
# Mudar formatos

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
    'body_type': 'string',
    'mileage': float,
})

In [236]:
novo_valor = df['mileage'].mean()
df['mileage'] = df['mileage'].fillna(novo_valor)

In [237]:
novo_valor = df['engine_capacity'].mean()
df['engine_capacity'] = df['engine_capacity'].fillna(novo_valor)

In [238]:
novo_valor = df['max_power'].mean()
df['max_power'] = df['max_power'].fillna(novo_valor)

In [239]:
novo_valor = df['kms_driven'].mean()
df['kms_driven'] = df['kms_driven'].fillna(novo_valor)

In [240]:
moda_seats = df['seats'].mode()[0]
df['seats'] = df['seats'].fillna(moda_seats)

In [241]:
moda_seats = df['owner_type'].mode()[0]
df['owner_type'] = df['owner_type'].fillna(moda_seats)

In [242]:
# Change formats

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
   
})

In [243]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['modelo'], dft['categoria_ano']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['categoria_ano'] = df['modelo'].map(mapeamento_max_power_group)

In [244]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['categoria_ano'], dft['id_modelos']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_modelos'] = df['modelo'].map(mapeamento_max_power_group)

In [245]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_id_modelos = dft['id_modelos'].max()
    print(maior_valor_id_modelos)
    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_modelos + 0.1
    print(novo_valor)
    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_modelos'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

4.130591051963374
4.230591051963374
                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0          1197.00000  Compre

In [246]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['transmission_type'], dft['id_transmissao']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_transmissao'] = df['transmission_type'].map(mapeamento_max_power_group)

In [247]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['marca'], dft['categoria_marca']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['categoria_marca'] = df['marca'].map(mapeamento_max_power_group)

In [248]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['categoria_marca'], dft['id_categoria_marca']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_categoria_marca'] = df['categoria_marca'].map(mapeamento_max_power_group)

In [249]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_categoria_marca'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_categoria_marca'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [250]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['body_type'], dft['id_body_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_body_type'] = df['body_type'].map(mapeamento_max_power_group)

In [251]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_body_type'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [252]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['city'], dft['id_city']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_city'] = df['city'].map(mapeamento_max_power_group)

In [253]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['insurance'], dft['id_insurance']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_insurance'] = df['insurance'].map(mapeamento_max_power_group)

In [254]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['fuel_type'], dft['id_fuel_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_fuel_type'] = df['fuel_type'].map(mapeamento_max_power_group)

In [255]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['owner_type'], dft['id_owner_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_owner_type'] = df['owner_type'].map(mapeamento_max_power_group)

In [256]:
df = df.astype({
    
    'ano': float,
  
    
  
})

In [257]:
contagem_nulos = df.isnull().sum()

print(f"O número de valores nulos na coluna é: {contagem_nulos}")

O número de valores nulos na coluna é: full_name               0
registered_year        15
engine_capacity         0
insurance               0
transmission_type       0
kms_driven              0
owner_type              0
fuel_type               0
max_power               0
seats                   0
mileage                 0
body_type               0
city                    0
ID                      0
ano                     0
marca                   0
modelo                 73
categoria_ano         389
id_modelos              0
id_transmissao          0
categoria_marca        73
id_categoria_marca      0
id_body_type            0
id_city                 0
id_insurance            0
id_fuel_type            0
id_owner_type           0
dtype: int64


In [258]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [259]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,...,categoria_ano,id_modelos,id_transmissao,categoria_marca,id_categoria_marca,id_body_type,id_city,id_insurance,id_fuel_type,id_owner_type
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.00000,Comprehensive,Manual,53658.0,First Owner,Petrol,81.8,5.0,...,muito baixa,4.230591,0.130634,baixa,0.269535,0.363138,0.923765,0.446914,0.180811,0.151316
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.00000,Not Available,Automatic,33055.0,Second Owner,Petrol,81.8,5.0,...,baixa,4.230591,0.585160,muito baixa,0.514641,0.363138,1.280558,1.414588,0.180811,0.619644
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.00000,Third Party,Manual,90385.0,Third Owner,Petrol,67.1,5.0,...,muito baixa,4.230591,0.130634,muito baixa,0.514641,0.363138,1.594033,0.256167,0.180811,1.355345
3,2010 Maruti Alto LXi,2010,796.00000,Third Party,Manual,110000.0,First Owner,Petrol,46.3,5.0,...,muito baixa,4.230591,0.130634,muito baixa,0.514641,0.363138,0.768863,0.256167,0.180811,0.151316
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.00000,Comprehensive,Manual,58000.0,Second Owner,Petrol,81.8,5.0,...,baixa,4.230591,0.130634,baixa,0.269535,0.363138,0.768863,0.446914,0.180811,0.619644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,1422.99627,Third Party,Automatic,30000.0,First Owner,Electric,73.7,5.0,...,média-baixa,4.230591,0.585160,baixa,0.269535,0.577680,1.095161,0.256167,2.458493,0.151316
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.00000,Third Party,Manual,64024.0,First Owner,Petrol,67.0,5.0,...,muito baixa,4.230591,0.130634,baixa,0.269535,0.363138,0.868615,0.256167,0.180811,0.151316
3487,2017 Honda City i VTEC VX,Dec 2017,1497.00000,Comprehensive,Manual,39000.0,First Owner,Petrol,117.3,5.0,...,baixa,4.230591,0.130634,baixa,0.269535,0.577680,0.768863,0.446914,0.180811,0.151316
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.00000,Comprehensive,Manual,33000.0,First Owner,Petrol,88.7,5.0,...,baixa,4.230591,0.130634,baixa,0.269535,0.363138,0.923765,0.446914,0.180811,0.151316


In [260]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID', 'ano',
       'marca', 'modelo', 'categoria_ano', 'id_modelos', 'id_transmissao',
       'categoria_marca', 'id_categoria_marca', 'id_body_type', 'id_city',
       'id_insurance', 'id_fuel_type', 'id_owner_type'],
      dtype='object')

In [261]:


# Obter valores únicos na coluna 'seats'
valores_unicos_seats = df['seats'].unique()

print(valores_unicos_seats)


[ 5.  7.  6.  8.  4.  9. 14.  2.]


In [262]:
valores_nulos = df.isnull().sum()
print(valores_nulos)

full_name               0
registered_year        15
engine_capacity         0
insurance               0
transmission_type       0
kms_driven              0
owner_type              0
fuel_type               0
max_power               0
seats                   0
mileage                 0
body_type               0
city                    0
ID                      0
ano                     0
marca                   0
modelo                 73
categoria_ano         389
id_modelos              0
id_transmissao          0
categoria_marca        73
id_categoria_marca      0
id_body_type            0
id_city                 0
id_insurance            0
id_fuel_type            0
id_owner_type           0
dtype: int64


In [263]:
#Export
#Not the target
colunas_selecionadas = ['id_transmissao', 'id_body_type','id_modelos','id_categoria_marca', 'mileage','ano','id_city','id_insurance','seats','engine_capacity','id_fuel_type','kms_driven','max_power','id_owner_type','ID']

if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('Tes11-12-2023V2.csv', index=False)